<a href="https://colab.research.google.com/gist/netcatt/f4176aed42caf92d2a14a828739164e9/voice-cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure GPU is enabled
Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU

In [10]:
%tensorflow_version 1.x

In [11]:
# Clone git repo
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2514 (delta 0), reused 1 (delta 0), pack-reused 2511
Receiving objects: 100% (2514/2514), 360.80 MiB | 39.46 MiB/s, done.
Resolving deltas: 100% (1390/1390), done.


In [12]:
%cd /content/Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


In [13]:
# Install dependencies
!pip install -q -r requirements.txt
!pip install webrtcvad
!apt-get install -qq libportaudio2

In [14]:
# Download dataset
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc -O /content/Real-Time-Voice-Cloning/pretrained.zip
!unzip -o /content/Real-Time-Voice-Cloning/pretrained.zip -d /content/Real-Time-Voice-Cloning/
!rm /content/Real-Time-Voice-Cloning/pretrained.zip

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:07, 50.2MB/s]
Archive:  /content/Real-Time-Voice-Cloning/pretrained.zip
  inflating: /content/Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt  
 extracting: /content/Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: /content/Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: /content/Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: /content/Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
  inflating: /content/Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt  


In [18]:
!wget https://yt-dl.org/downloads/latest/youtube-dl -O /usr/local/bin/youtube-dl
!chmod a+rx /usr/local/bin/youtube-dl
!rm -f voice.wav voice.mp3
!youtube-dl -x --audio-format mp3 https://www.youtube.com/watch?v=m9aGBKxDx6E -o /content/Real-Time-Voice-Cloning/voice.mp3
!ffmpeg -i /content/Real-Time-Voice-Cloning/voice.mp3 /content/Real-Time-Voice-Cloning/voice.wav
!rm /content/Real-Time-Voice-Cloning/voice.mp3

--2020-08-14 00:59:48--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2020.07.28/youtube-dl [following]
--2020-08-14 00:59:48--  https://yt-dl.org/downloads/2020.07.28/youtube-dl
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ytdl-org/youtube-dl/releases/download/2020.07.28/youtube-dl [following]
--2020-08-14 00:59:49--  https://github.com/ytdl-org/youtube-dl/releases/download/2020.07.28/youtube-dl
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github-production-release-asset-2e65be.s3.amazonaws.com/1039520/379b

In [19]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("/content/Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt")
vocoder_weights = Path("/content/Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("/content/Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at /content/Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt


In [ ]:
in_fpath = Path("voice.wav")
print("Started preprocessing")
reprocessed_wav = encoder.preprocess_wav(in_fpath)
print("Finished preprocessing")
original_wav, sampling_rate = librosa.load(in_fpath)
print("Loaded file")
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
print("Preoprocessed with original sample rate")
embed = encoder.embed_utterance(preprocessed_wav)
print("finished embeding utterance")

Started preprocessing


In [ ]:
text = "macarena"
print("Synthesizing new audio...")
with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
generated_wav = vocoder.infer_waveform(specs[0])
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
display(Audio(generated_wav, rate=synthesizer.sample_rate))